In [1]:
import pandas as pd
import wittgenstein as lw
print('Importing Wittgenstein Version', lw.__version__)
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import time
# from wittgenstein.base import Timer
# import pickle

This is a local version of Wittgenstein
Importing Wittgenstein Version 0.1.6


In [2]:
def convert_label(label):
    return (label == ' false') # ' false', ' true'

In [9]:
df = pd.read_csv('data/miniloan-decisions-100K.csv') # miniloan-decisions-100K.csv, miniloan-adjusted-and-enriched.csv
# del df['yearlyReimbursement'] # Is this derived from the input?
df['approval'] = df['approval'].map(convert_label) # label needs Boolean or {0,1}
print('Label distribution')
print(df['approval'].value_counts())
df['ratio'] = df['yearlyReimbursement'] / df['income']
# df['ratio'] = round(df['yearlyReimbursement'] / df['income'], 2)
# df['ratio'] = 100 * round(df['yearlyReimbursement'] / df['income'], 2)
# df['ratio'] = df['ratio'].map(int)
# df['ratio_print'] = df['ratio'].map(str)

# df = df[df['monthDuration'] > 0]
df.info()

# Check hypothesis that data is generated from the following two rules
df_from_rule = df[((df['ratio'] < 0.3) & (df['loanAmount'] < 1000000)) == (df['approval'])]
df_from_rule.info()
# df_residue = df[(df['ratio'] < 0.3) & (df['loanAmount'] < 1000000) & ~df['approval']]
# df_residue.info()
# df_residue = df[(df['loanAmount'] >= 1000000) & df['approval']]
# df_residue.info()
# df_residue = df[(df['ratio'] >= 0.3) & df['approval']]
# df_residue.info()
# df_residue.head(5)
# print(df['ratio'].value_counts())
# df.to_csv('out_miniloan.csv', index=False)

Label distribution
True     78046
False    21954
Name: approval, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
name                   100000 non-null object
creditScore            100000 non-null int64
income                 100000 non-null int64
loanAmount             100000 non-null int64
monthDuration          100000 non-null int64
rate                   100000 non-null float64
approval               100000 non-null bool
yearlyReimbursement    100000 non-null int64
ratio                  100000 non-null float64
dtypes: bool(1), float64(2), int64(5), object(1)
memory usage: 6.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 9 columns):
name                   0 non-null object
creditScore            0 non-null int64
income                 0 non-null int64
loanAmount             0 non-null int64
monthDuration          0 non-null int64
rate                   0 non-null float6

In [10]:
# IGNORE THIS
# df2 = df[(df['ratio'] >= 0.29994492856911464) & (df['ratio'] <= 0.2999544476383457)]
# df2 = df[(df['ratio'] >= 0.299944928569114) & (df['ratio'] <= 0.29995444763835)]
# df2
# df3 = df[(df['loanAmount'] >= 999609) & df['approval']]
# df3

,name,creditScore,income,loanAmount,monthDuration,rate,approval,yearlyReimbursement,ratio
1,John Doe,527,224186,1787385,183,0.054819,True,173197,0.772559
2,John Doe,460,39954,1733494,75,0.055183,True,328563,8.223532
3,John Doe,751,219998,1350004,72,0.043400,True,255970,1.163511
4,John Doe,684,72470,1210944,160,0.037772,True,115742,1.597102
8,John Doe,822,59518,1724869,299,0.042252,True,112054,1.882691
...,...,...,...,...,...,...,...,...,...
99991,John Doe,483,133020,1847003,206,0.042775,True,152069,1.143204
99995,John Doe,366,83851,1718571,168,0.053737,True,174928,2.086177
99996,John Doe,395,184259,1599473,90,0.040564,True,247703,1.344320
99997,John Doe,674,242620,1099790,260,0.040825,True,76557,0.315543


In [5]:
train, test = train_test_split(df, test_size=.33, random_state=42)
# ripper_clf = lw.RIPPER(k=3, prune_size=0.33, dl_allowance=32, n_discretize_bins=4, verbosity=0) # n_discretize_bins=None, max_rules=2

# dir(ripper_clf)

In [6]:
# BINS = 10
for BINS in [10,20,40]:
    print('BINS:', BINS)

    clf = lw.RIPPER(k=3, prune_size=0.33, dl_allowance=32, n_discretize_bins=BINS, verbosity=0) # n_discretize_bins=None, max_rules=2 

    start_time = time.time()
    print('Job started at {}.'.format(time.ctime()))

    clf.fit(train, class_feat='approval') # Or pass X and y data to .fit , initial_model='[ratio=0.13-0.3]'
    end_time = time.time()
    elapsed = end_time - start_time
    print('Job finished at {} in {} seconds.'.format(time.ctime(), round(elapsed, 1)))

    X_test = test.drop('approval', axis=1)
    y_test = test['approval']
    print('accuracy: ', clf.score(X_test, y_test))

    # precision = clf.score(X_test, y_test, precision_score)
    # recall = clf.score(X_test, y_test, recall_score)
    # print(f'precision: {precision} recall: {recall}')

    clf.out_model()
    print('')
    clf

BINS: 10
Job started at Tue Jun  8 16:30:40 2021.
Job finished at Tue Jun  8 16:31:21 2021 in 41.0 seconds.
accuracy:  0.8797575757575757
[[yearlyReimbursement=196877-350189] V
[yearlyReimbursement=145522-196877] V
[yearlyReimbursement=98657-119330] V
[yearlyReimbursement=350189-24010593] V
[yearlyReimbursement=119330-145522] V
[yearlyReimbursement=78769-98657] V
[yearlyReimbursement=58779-78769 ^ loanAmount=998258-1198414] V
[yearlyReimbursement=58779-78769 ^ ratio=0.39-0.54] V
[income=8-29803 ^ ratio=2.53-5.98] V
[ratio=0.54-0.73] V
[income=29803-60281 ^ ratio=1.02-1.5] V
[income=8-29803 ^ ratio=5.98-294928.88] V
[ratio=0.26-0.39 ^ loanAmount=1198414-1397900] V
[ratio=0.73-1.02] V
[ratio=0.26-0.39 ^ income=149807-179898 ^ yearlyReimbursement=58779-78769] V
[ratio=0.26-0.39 ^ income=119965-149807 ^ yearlyReimbursement=39073-58779 ^ loanAmount=793210-998258] V
[ratio=0.26-0.39 ^ income=119965-149807 ^ yearlyReimbursement=39073-58779 ^ loanAmount=597586-793210 ^ monthDuration=218-255] V

In [7]:
# BINS = 10
for BINS in [10,20,40]:
    print('BINS:', BINS)
    clf = lw.IREP(random_state=42, n_discretize_bins=BINS) 

    start_time = time.time()
    print('Job started at {}.'.format(time.ctime()))

    clf.fit(train, class_feat='approval') # Or pass X and y data to .fit , initial_model='[ratio=0.13-0.3]'
    end_time = time.time()
    elapsed = end_time - start_time
    print('Job finished at {} in {} seconds.'.format(time.ctime(), round(elapsed, 1)))

    X_test = test.drop('approval', axis=1)
    y_test = test['approval']
    print('accuracy: ', clf.score(X_test, y_test))

    # precision = clf.score(X_test, y_test, precision_score)
    # recall = clf.score(X_test, y_test, recall_score)
    # print(f'precision: {precision} recall: {recall}')

    clf.out_model()
    print('')

    clf

BINS: 10
Job started at Tue Jun  8 16:33:42 2021.
Job finished at Tue Jun  8 16:33:48 2021 in 6.4 seconds.
accuracy:  0.8848787878787879
[[ratio=1.02-1.5] V
[ratio=1.5-2.53] V
[ratio=2.53-5.98] V
[ratio=0.54-0.73] V
[ratio=0.73-1.02] V
[ratio=5.98-294928.88] V
[ratio=0.39-0.54] V
[ratio=0.26-0.39] V
[loanAmount=998258-1198414] V
[loanAmount=1198414-1397900] V
[monthDuration=0-37 ^ loanAmount=1597029-1796839] V
[loanAmount=1397900-1597029] V
[loanAmount=1796839-1999979]]

BINS: 20
Job started at Tue Jun  8 16:33:50 2021.
Job finished at Tue Jun  8 16:33:59 2021 in 8.6 seconds.
accuracy:  0.9731515151515151
[[ratio=1.5-1.9] V
[ratio=1.02-1.22] V
[ratio=0.54-0.62] V
[ratio=0.73-0.86] V
[ratio=0.39-0.46] V
[ratio=1.9-2.53] V
[ratio=6.0-13.63] V
[ratio=0.46-0.54] V
[ratio=1.22-1.5] V
[ratio=0.62-0.73] V
[ratio=0.33-0.39] V
[ratio=3.64-6.0] V
[ratio=2.53-3.64] V
[ratio=0.86-1.02] V
[ratio=13.63-294928.88] V
[ratio=0.26-0.33 ^ yearlyReimbursement=78785-88768] V
[ratio=0.26-0.33 ^ yearlyReimbu

In [8]:
# for i in range(200):
#     ripper_clf.remove_rule_at(0)
# print(len(ripper_clf.ruleset_))
new_rule_set = '[[ratio=0.1-0.3] V [ratio=0.15-0.17 ^ yearlyReimbursement=0-2103]]'
ripper_clf.set_ruleset(new_rule_set)
ripper_clf.out_model()
print('accuracy: ', ripper_clf.score(X_test, y_test))
# df['predict'] = ripper_clf.predict(df)
# test.to_csv('out.csv') # , index=False
# precision = ripper_clf.score(X_test, y_test, precision_score)
# recall = ripper_clf.score(X_test, y_test, recall_score)
# print(f'precision: {precision} recall: {recall}')



NameError: name 'ripper_clf' is not defined

In [56]:
ripper_2 = lw.RIPPER() # n_discretize_bins=None, max_rules=2
ripper_2.init_ruleset(new_rule_set)
ripper_2.out_model()
df['predict'] = ripper_2.predict(df)
df.to_csv('out.csv', index=False)
# print('accuracy: ', ripper_2.score(X_test, y_test))


[[ratio=0.1-0.3] V
[ratio=0.15-0.17 ^ yearlyReimbursement=0-2103]]


AttributeError: 'RIPPER' object has no attribute 'bin_transformer_'